In [11]:
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv()


True

In [12]:
from langchain_openai import ChatOpenAI
from langchain_teddynote import logging
from langchain_teddynote.messages import stream_response
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import FewShotChatMessagePromptTemplate
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_community.vectorstores.utils import DistanceStrategy
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.document_loaders import TextLoader
from langchain_community.document_loaders import UnstructuredMarkdownLoader
from langchain_community.document_loaders import UnstructuredExcelLoader
import pandas
from llama_parse import LlamaParse

In [13]:
# load and text split

loader = TextLoader('2-2. 졸업이수학점 안내_Aspose.md', encoding = 'utf-8')
data = loader.load()
splitter = CharacterTextSplitter(
    separator = '//<>',
    # chunk_size = 4500,
    # chunk_overlap = 500,
    # length_function = len
)

documents = splitter.split_documents(data)
len(documents)

Created a chunk of size 4542, which is longer than the specified 4000
Created a chunk of size 6428, which is longer than the specified 4000
Created a chunk of size 9656, which is longer than the specified 4000
Created a chunk of size 8983, which is longer than the specified 4000
Created a chunk of size 9209, which is longer than the specified 4000
Created a chunk of size 7541, which is longer than the specified 4000


8

In [14]:
# save embeddings in vectorstore

embed_model = HuggingFaceEmbeddings(
    model_name = 'jhgan/ko-sroberta-nli',
    model_kwargs = {'device':'cpu'},
    encode_kwargs = {'normalize_embeddings':True}
)

vectorstore = FAISS.from_documents(documents,
                                   embedding = embed_model,
                                   distance_strategy = DistanceStrategy.COSINE)

c:\Users\jw\.pyenv\pyenv-win\versions\3.11.9\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [15]:

llm = ChatOpenAI(
    temperature = 0,
    model_name = "gpt-4o-mini",
    max_tokens=500
)

In [18]:
# query and retirever

query = '2024학년도 소프트웨어학부 신입학자가 졸업하기 위한 학점을 알려줘.'

retriever = vectorstore.as_retriever(search_kwargs={'k': 2})

docs = retriever.get_relevant_documents(query)

for i in docs:
    print(i.page_content, "\n")

`   `타.  2024학년도  신입학자

\1)  졸업이수학점(2025학년도  2학년  편입생,  2026학년도  3학년  편입생  포함)



<table><tr><th colspan="2" rowspan="2">단과대</th><th colspan="2">교양</th><th colspan="2">주전공학점 (필수포함)</th><th colspan="1" rowspan="1">심화 전공</th><th colspan="1" rowspan="1">복수 전공</th><th colspan="1" rowspan="2">부전공</th><th colspan="1" rowspan="2">졸업이수 학점</th></tr>
<tr><td colspan="1">필수</td><td colspan="1">균형</td><td colspan="1">단일 전공시</td><td colspan="1">다전공 이수시＊</td></tr>
<tr><td colspan="2">전자정보공과대학</td><td colspan="1" rowspan="2">13</td><td colspan="1" rowspan="7">18</td><td colspan="1">60</td><td colspan="1">54</td><td colspan="1">-</td><td colspan="1">54</td><td colspan="1">21</td><td colspan="2">133</td></tr>
<tr><td colspan="2">인공지능융합대학</td><td colspan="1">60</td><td colspan="1">54</td><td colspan="1">-</td><td colspan="1">54</td><td colspan="1">21</td><td colspan="2">133</td></tr>
<tr><td colspan="1" rowspan="2">공과대학</td><td colspan="1">건축공,화공,환경</td><td colspan="1">60</td><td colspan="1">54</td><td 

In [17]:
# generation


template = '''당신은 광운대학교 학생인 사용자에게 사용자가 입력한 자신의 입학 연도를 기반으로 졸업 요건을 알려주는 인공지능 챗봇입니다.
            졸업 이수학점 표는 총 7열입니다. 이때 7열중 교양은 (필수+균형)과 기초라는 2열로 다시 나누어집니다.
            주전공학점(필수 포함)은 단일 전공시와 다전공 이수시로 2열로 나누어집니다.
            표에서 이웃한 여러 셀이 동일한 값을 가질 경우 그 열의 같은 값을 가지는 이웃한 셀들을 통합하기도 합니다.
            또한 이 문서는 각 졸업 요건이 동일한 입학 연도 단위 앞 부분에 가., 나., 다. ... 순으로 번호를 매깁니다.
            이 중 어떤 부분을 참고해서 답변 하였는지 맨 처음에 명시하세요.
            오직 한가지 한 단위만 참고해서 답변해야 하며, 당신이 참고했다 말하는 부분만을 기반으로 답변해야 합니다.
            생략되는 정보가 없어야 합니다.
            만약 표에 나타나 있는 데이터를 참고해야 한다면, 표 내용을 그대로 출력하되, 병합된 셀이 있는지를 감안하세요요.
            오직 다음의 context에 기반하여 대답하세요. {context}, Question: {question}
            '''

prompt = ChatPromptTemplate.from_template(template)

# 이 부분은 뭔지 모르겠음
# def format_docs(docs):
#     return '\n\n'.join([d.page_content for d in docs])

# Chain
chain = prompt | llm | StrOutputParser()

# Run
response = chain.invoke({'context': docs, 'question':query})
print(response)

저는 "카. 2020학년도 신입학 ~ 2023학년도 신입학자" 부분을 참고하여 답변하겠습니다.

2024학년도 소프트웨어학부 신입학자가 졸업하기 위한 학점은 다음과 같습니다:

- **교양**: 필수 + 균형 22학점, 기초 24학점
- **주전공학점 (필수 포함)**: 단일 전공시 60학점, 다전공 이수시 54학점
- **심화 전공**: -
- **복수 전공**: 54학점
- **부전공**: 21학점
- **졸업이수 학점**: 133학점

이 정보를 바탕으로 졸업 요건을 충족하시기 바랍니다.
